In [1]:
!pip3 install ax-platform
!pip3 install botorch

In [2]:
class Data:
    def __init__(self, sensorPositions, space, epsilon):
        self.radius = 1
        self.placeHolders = sensorPositions
        self.epsilon = epsilon
        self.space = space
        # self.SensorPlaceHolderSetup()
        
    def frange(self, start, stop, step):
        steps = []
        while start <= stop:
            steps.append(start)
            start +=step
            
        return steps

    def GetSensorConfiguration(self):
        from collections import Counter
        sensorLocations, sensorTypes = self.GetSensorLocations()

        summaryDict = Counter(sensorTypes)

        # TODO: DIFFERENT SENSOR TYPE DEFINITIONS SHOULD BE ADDED HERE:
        configurationSummary = []
        for key in summaryDict:
            if (key == 1):
                configurationSummary.append(['motion sensors', summaryDict[key]])

            elif (key == 2):
                configurationSummary.append(['beacon sensors', summaryDict[key]])

        configurationDetails = []
        for index, loc in enumerate(sensorLocations):
            if (sensorTypes[index] == 1):
                configurationDetails.append(tuple([loc, 'kitchen', 'motion sensors']))

            elif (sensorTypes[index] == 2):
                configurationDetails.append(tuple([loc, 'kitchen', 'beacon sensors']))

        return [[configurationSummary, [tuple(configurationDetails)]], self.radius]


    def GetSensorLocations(self):
        sensorLocations = []
        sensorTypes = []
        for index, sensorIndicator in enumerate(self.placeHolders):
            sensorLocations.append(self.placeHolders[index])

            # TODO: DIFFERENT SENSOR TYPE DEFINITIONS SHOULD BE ADDED HERE:
            sensorTypes.append(1)


        return sensorLocations, sensorTypes


class BOVariables:
    def __init__(self, Data_path, epsilon, initSensorNum, maxSensorNum, radius, sampleSize):
        self.epsilon = epsilon
        self.Data_path = Data_path
        self.initSensorNum = initSensorNum
        self.maxSensorNum = maxSensorNum
        self.radius = radius
        self.sensor_distribution, self.types, self.space, self.rooms, self.agentTraces = self.ModelsInitializations()

    def ModelsInitializations(self):
        #----- Space and agent models -----: 
        simworldname = self.Data_path + '/Configuration Files/simulationWorld2.xml'
        agentTraces = []
        directory = os.fsencode(self.Data_path + 'Agent Trace Files2/')
            
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".csv"): 
                agentTraces.append(self.Data_path + 'Agent Trace Files2/' + filename)

        # Parsing the space model: 
        space, rooms = pf.ParseWorld(simworldname)

        xs = []
        for i in space:
          for j in i:
            xs.append(j)
        A = list(set(xs))
        A.sort()
        space = [A[-1], A[-2]]

        # User parameters 
        types, sensor_distribution = pf.GetUsersParameters()

        roomsList = []
        for room in sensor_distribution:
            roomsList.append(room)
              
        return sensor_distribution, types, space, rooms, agentTraces


def frange(start, stop, step):
    steps = []
    while start <= stop:
        steps.append(start)
        start +=step
        
    return steps

def MakeSensorCombinations(start, end, epsilon, sensorType, room):
    a1, b1 = makeBoundaries(epsilon, start[0], end[0])
    a2, b2 = makeBoundaries(epsilon, start[1], end[1])    
    Xs = frange(a1, b1, epsilon)
    Ys = frange(a2, b2, epsilon)
    
    points = list(itertools.product(list(itertools.product(Xs, Ys)), [room], [sensorType[0]])) 
    C = itertools.combinations(points, distribution[room][types.index(sensorType)])

    return C

def PreProcessor(df):
    df['motion sensors'] = df['motion sensors'].apply(lambda s: list(map(int, s)))
    try:
      df['beacon sensors'] = df['beacon sensors'].apply(lambda s: list(map(int, s)))
    except:
      pass

    sensors = set([])

    previous_M = None
    previous_B = None
    output_file = []

    for index, row in df.iterrows():
      T = row['time']
      M = row['motion sensors']
      try:
        B = row['beacon sensors']
      except:
        pass

      Activity = row['activity']
      Activity = Activity.replace(' ', '_')
      MotionSensor_Names = []
      sensorNames = []
      MotionSensor_Message = []
      BeaconSensor_Names = []
      BeaconSensor_Message = []
      

      # time = convertTime(T)
      time = "2020-06-16 " + T + ".00"

      # Motion Sensor
      for i in range(len(M)):
        sensorNames.append(Name(i, 'M'))
        if M[i] == 1:
          if (previous_M != None):
            if (previous_M[i] == 0):
              MotionSensor_Names.append(Name(i,'M'))
              MotionSensor_Message.append('ON')

          else:
            MotionSensor_Names.append(Name(i,'M'))
            MotionSensor_Message.append('ON')

        if previous_M != None:
          if M[i] == 0 and previous_M[i] == 1:
            MotionSensor_Names.append(Name(i,'M'))
            MotionSensor_Message.append('OFF')

      previous_M = M
      # Beacon Sensor

      try:
        for i in range(len(B)):
          sensorNames.append(Name(i, 'B'))
          if B[i] == 1:
            BeaconSensor_Names.append(Name(i,'B'))
            BeaconSensor_Message.append('ON')
          if previous_B != None:
            if B[i] == 0 and previous_B[i] == 1: 
              BeaconSensor_Names.append(Name(i,'B'))
              BeaconSensor_Message.append('OFF')
        previous_B = B

      except:
        pass

      for m in range(len(MotionSensor_Names)):
        output_file.append(time +' '+ MotionSensor_Names[m] + ' ' + MotionSensor_Names[m] + ' ' + MotionSensor_Message[m] + ' ' + Activity)
        
      for s in sensorNames:
          sensors.add(s)

    return output_file, list(sensors)

#returns the name of the sensor
def Name(number, typeSensor):
    if number < 10:
      return typeSensor + str(0) + str(number)
    else:
      return typeSensor + str(number)

#converts epoch time to human readable
def convertTime(posix_timestamp):
    tz = pytz.timezone('MST')
    dt = datetime.fromtimestamp(posix_timestamp, tz)
    time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return time

def MakeDataBoundaries(height = 10.5, width = 6.6, MaxSensors = 15):
    from collections import defaultdict, OrderedDict

    d = dict()

    for idx in range(MaxSensors):
            d['x' + str(idx)] = (0.5, width - 0.5)
            d['y' + str(idx)] = (0.5, height - 0.5)

    return d

def black_box_function(sample, simulateMotionSensors = True, simulateEstimotes = False, Plotting = False):       
    files = []

    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
        
    else:
        sys.path.append('../../Codes/Sensor Simulator/')
        Data_path = '../../Codes/Sensor Simulator/'

    all_sensors = set([])

    for agentTrace in BOV.agentTraces:
        df_ = sim_sis.RunSimulator(BOV.space, BOV.rooms, agentTrace, sample.GetSensorConfiguration(), simulateMotionSensors, simulateEstimotes, Plotting, BOV.Data_path)
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        files.append(dataFile)
        
    
    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')

    else:
        sys.path.append('../../Codes/CASAS/AL-Smarthome')

    import al
    import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    accuracy = (al.leave_one_out(files, all_sensors)[0]) * 100

    if accuracy < 0:
        accuracy = 0

    return [accuracy]

def function_to_be_optimized(T):
    sensorPositions = []
    sensor_xy = []
    for i, item in enumerate(T):
            if i % 2 == 0:
                sensor_xy.append(item)

            else:
                sensor_xy.append(item)  
                sensorPositions.append(sensor_xy)
                sensor_xy = []

    data = Data(sensorPositions, BOV.space, CONSTANTS['epsilon'])
    return torch.Tensor(black_box_function(data))

def InitializeDataset(BOV, init_num = 5):
    train_x = []
    for i in range(init_num):
        Xs = frange(0, BOV.space[0], BOV.epsilon)
        Ys = frange(0, BOV.space[1], BOV.epsilon)
        grid = np.zeros(len(Xs) * len(Ys)).tolist()
        placeHolders = []


        for x in Xs:
            for y in Ys:
                placeHolders.append([x, y])

        i = 0
        while i < BOV.initSensorNum:
            cell = random.randrange(len(grid))
            if grid[cell] == 0:
                grid[cell] = 1
                i += 1

        sensorPositions = []
        sensorTypes = []
        for index, sensorIndicator in enumerate(grid):
            if (sensorIndicator > 0):
                # sensorPositions.append(torch.Tensor(placeHolders[index]))
                sensorPositions.append(placeHolders[index][0])
                sensorPositions.append(placeHolders[index][1])
                sensorTypes.append(torch.Tensor(sensorIndicator))


        sensorPositions = torch.Tensor(sensorPositions)


        # data = Data(sensorPositions, BOV.space, CONSTANTS['epsilon'])
        train_x.append(sensorPositions)

    train_x = torch.stack(train_x)

    train_y = []

    sensorPositions = []
    sensor_xy = []
    for T in train_x:

        for i, item in enumerate(T):
            if i % 2 == 0:
                sensor_xy.append(item)

            else:
                sensor_xy.append(item)  
                sensorPositions.append(sensor_xy)
                sensor_xy = []

        data = Data(sensorPositions, BOV.space, CONSTANTS['epsilon'])
        train_y.append(torch.Tensor(black_box_function(data)))

    train_y = torch.stack(train_y)
    best_observed_value = train_y.max()

    return train_x, train_y, best_observed_value
    

def frange(start, stop, step):
    steps = []
    while start <= stop:
        steps.append(start)
        start +=step
        
    return steps

def get_next_points(train_x, train_y, best_observed_value, bounds, n_points = 10):
    single_model = SingleTaskGP(train_x, train_y)
    mll = ExactMarginalLogLikelihood(single_model.likelihood, single_model)
    fit_gpytorch_model(mll)
    
    print('----- Calculating Next Candidates Point From the Posterior PDF ...', end = " ")
    MES = qMaxValueEntropy(
                           model = single_model,
                           best_f = best_observed_value,
                           candidate_set = train_x
                          )
    

    candidates, _ = optimize_acqf(
                                  acq_function = MES,
                                  bounds = bounds,
                                  q = n_points, # number or candidates to return
                                  num_restarts = CONSTANTS['restarts_number'],
                                  raw_samples = CONSTANTS['raw_samples'],
                                  # options = {"batch_limit": 5, "maxiter": 100}
                                  sequential = CONSTANTS['sequential']
                                 )
    print('[COMPLETED]')

    return candidates

In [ ]:
CONSTANTS = {
    'iterations': 100,
    'initial_samples': 50,
    'restarts_number': 15,
    'raw_samples': 15,
    'candidates_num': 20,
    'sequential': True,
    'epsilon': 1,
    'radius': 1,
    'height': 10.5,
    'width': 6.6,
    'max_sensors': 15  
}

global runningOnGoogleColab
runningOnGoogleColab = False
    
if __name__ ==  '__main__':
    import sys

    if (runningOnGoogleColab == True):
        from google.colab import drive    
        drive.mount('/content/gdrive', force_remount=True)
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
        
    else:
        Data_path = '../../Codes/Sensor Simulator/'
        sys.path.append('../../Codes/Sensor Simulator/')

    # from ax import optimize, ChoiceParameter
    from scipy.stats import norm
    from numpy import argmax
    import SIM_SIS_Libraries.SensorsClass
    import SIM_SIS_Libraries.SIM_SIS_Simulator as sim_sis
    import SIM_SIS_Libraries.ParseFunctions as pf
    import itertools
    import numpy as np
    import pandas as pd
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde
    import copy
    from datetime import datetime
    import pytz
    import ast
    import os
    import random
    import plotly
    import torch

    from botorch.models import SingleTaskGP, ModelListGP
    from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
    from botorch import fit_gpytorch_model
    from botorch.acquisition.max_value_entropy_search import qMaxValueEntropy
    from botorch.optim import optimize_acqf

    # epsilon = 1
    # radius = 1
    finalResults = []
    # height = 10.5
    # width = 6.6
    # MaxSensors = 15
    w = CONSTANTS['width'] - 0.5
    h = CONSTANTS['height'] - 0.5
    # n_runs = 100
    # init_num = 10

    dataBoundaries = MakeDataBoundaries(
                                        height = CONSTANTS['height'], 
                                        width = CONSTANTS['width'], 
                                        MaxSensors = CONSTANTS['max_sensors']
                                       )

    BOV =  BOVariables(
                       Data_path, 
                       CONSTANTS['epsilon'], 
                       CONSTANTS['max_sensors'], 
                       CONSTANTS['max_sensors'], 
                       CONSTANTS['radius'],
                       CONSTANTS['initial_samples']
                      )
    
    print('----- Initializing Dataset... ', end = " ")
    train_x, train_y, best_observed_value = InitializeDataset(
                                                              BOV, 
                                                              init_num = CONSTANTS['initial_samples']
                                                             )
    
    print('[COMPLETED]')
    
    bounds = torch.tensor([[0.5] * CONSTANTS['max_sensors'] * 2, [w, h] * CONSTANTS['max_sensors']])

    for i in range(CONSTANTS['iterations']):
        print(f"Optimization iteration: {i + 1}")
        new_candidates = get_next_points(
                                         train_x, 
                                         train_y, 
                                         best_observed_value, 
                                         bounds, 
                                         n_points = CONSTANTS['candidates_num']
                                        )
        
        for index, candidate in enumerate(new_candidates):
            print('\t * Accuracy of candidate #', index + 1, ' is: ', end = " ")
            new_result = function_to_be_optimized(candidate)
            train_x = torch.cat([train_x, candidate.unsqueeze(0)])
            train_y = torch.cat([train_y, torch.stack([new_result])])
            print(new_result[0])

        best_observed_value = train_y.max().item()
        print(f"\n\tBest Point Performs: {best_observed_value}")

----- Initializing Dataset...  [COMPLETED]
Optimization iteration: 1
----- Calculating Next Candidates Point From the Posterior PDF ... [COMPLETED]
	 * Accuracy of candidate # 1  is:  tensor(57.3052)
	 * Accuracy of candidate # 2  is:  tensor(41.7607)
	 * Accuracy of candidate # 3  is:  tensor(58.5392)
	 * Accuracy of candidate # 4  is:  tensor(46.0002)
	 * Accuracy of candidate # 5  is:  tensor(56.3829)
	 * Accuracy of candidate # 6  is:  tensor(50.1141)
	 * Accuracy of candidate # 7  is:  tensor(53.5295)
	 * Accuracy of candidate # 8  is:  tensor(49.1129)
	 * Accuracy of candidate # 9  is:  tensor(48.8422)
	 * Accuracy of candidate # 10  is:  tensor(48.1795)
	 * Accuracy of candidate # 11  is:  tensor(47.9241)
	 * Accuracy of candidate # 12  is:  tensor(47.6004)
	 * Accuracy of candidate # 13  is:  tensor(45.8227)
	 * Accuracy of candidate # 14  is:  tensor(57.0674)
	 * Accuracy of candidate # 15  is:  tensor(53.4114)
	 * Accuracy of candidate # 16  is:  tensor(53.0528)
	 * Accuracy 

	 * Accuracy of candidate # 17  is:  tensor(47.6022)
	 * Accuracy of candidate # 18  is:  tensor(49.1511)
	 * Accuracy of candidate # 19  is:  tensor(51.2765)
	 * Accuracy of candidate # 20  is:  tensor(49.1234)

	Best Point Performs: 58.53923416137695
Optimization iteration: 8
----- Calculating Next Candidates Point From the Posterior PDF ... [COMPLETED]
	 * Accuracy of candidate # 1  is:  tensor(51.8175)
	 * Accuracy of candidate # 2  is:  tensor(53.6192)
	 * Accuracy of candidate # 3  is:  tensor(48.1907)
	 * Accuracy of candidate # 4  is:  tensor(50.6468)
	 * Accuracy of candidate # 5  is:  tensor(48.3893)
	 * Accuracy of candidate # 6  is:  tensor(54.1090)
	 * Accuracy of candidate # 7  is:  tensor(42.3195)
	 * Accuracy of candidate # 8  is:  tensor(55.5712)
	 * Accuracy of candidate # 9  is:  tensor(51.1578)
	 * Accuracy of candidate # 10  is:  tensor(51.7655)
	 * Accuracy of candidate # 11  is:  tensor(52.1203)
	 * Accuracy of candidate # 12  is:  tensor(45.9488)
	 * Accuracy of

	 * Accuracy of candidate # 13  is:  tensor(47.8933)
	 * Accuracy of candidate # 14  is:  tensor(48.8930)
	 * Accuracy of candidate # 15  is:  tensor(48.0712)
	 * Accuracy of candidate # 16  is:  tensor(48.8813)
	 * Accuracy of candidate # 17  is:  tensor(50.0001)
	 * Accuracy of candidate # 18  is:  tensor(47.3970)
	 * Accuracy of candidate # 19  is:  tensor(49.6271)
	 * Accuracy of candidate # 20  is:  tensor(48.2194)

	Best Point Performs: 68.3356704711914
Optimization iteration: 15
----- Calculating Next Candidates Point From the Posterior PDF ... [COMPLETED]
	 * Accuracy of candidate # 1  is:  tensor(49.3130)
	 * Accuracy of candidate # 2  is:  tensor(45.6361)
	 * Accuracy of candidate # 3  is:  tensor(54.8113)
	 * Accuracy of candidate # 4  is:  tensor(41.3591)
	 * Accuracy of candidate # 5  is:  tensor(55.2405)
	 * Accuracy of candidate # 6  is:  tensor(51.0072)
	 * Accuracy of candidate # 7  is:  tensor(42.8600)
	 * Accuracy of candidate # 8  is:  tensor(54.1853)
	 * Accuracy o

	 * Accuracy of candidate # 9  is:  tensor(48.9231)
	 * Accuracy of candidate # 10  is:  tensor(47.4482)
	 * Accuracy of candidate # 11  is:  tensor(51.4261)
	 * Accuracy of candidate # 12  is:  tensor(48.6736)
	 * Accuracy of candidate # 13  is:  tensor(43.8797)
	 * Accuracy of candidate # 14  is:  tensor(44.4623)
	 * Accuracy of candidate # 15  is:  tensor(49.6282)
	 * Accuracy of candidate # 16  is:  tensor(51.1284)
	 * Accuracy of candidate # 17  is:  tensor(52.7995)
	 * Accuracy of candidate # 18  is:  tensor(56.7750)
	 * Accuracy of candidate # 19  is:  tensor(58.8971)
	 * Accuracy of candidate # 20  is:  tensor(47.7180)

	Best Point Performs: 68.3356704711914
Optimization iteration: 22
----- Calculating Next Candidates Point From the Posterior PDF ... [COMPLETED]
	 * Accuracy of candidate # 1  is:  tensor(47.8592)
	 * Accuracy of candidate # 2  is:  tensor(53.6635)
	 * Accuracy of candidate # 3  is:  tensor(59.0919)
	 * Accuracy of candidate # 4  is:  tensor(45.5753)
	 * Accurac

	 * Accuracy of candidate # 5  is:  tensor(45.2479)
	 * Accuracy of candidate # 6  is:  tensor(43.5844)
	 * Accuracy of candidate # 7  is:  tensor(51.9859)
	 * Accuracy of candidate # 8  is:  tensor(48.5540)
	 * Accuracy of candidate # 9  is:  tensor(48.2959)
	 * Accuracy of candidate # 10  is:  tensor(50.9414)
	 * Accuracy of candidate # 11  is:  tensor(50.3381)
	 * Accuracy of candidate # 12  is:  tensor(42.0542)
	 * Accuracy of candidate # 13  is:  tensor(51.6051)
	 * Accuracy of candidate # 14  is:  tensor(44.7147)
	 * Accuracy of candidate # 15  is:  tensor(60.0313)
	 * Accuracy of candidate # 16  is:  tensor(50.2949)
	 * Accuracy of candidate # 17  is:  tensor(47.9573)
	 * Accuracy of candidate # 18  is:  tensor(43.2885)
	 * Accuracy of candidate # 19  is:  tensor(49.1666)
	 * Accuracy of candidate # 20  is:  tensor(53.9511)

	Best Point Performs: 68.3356704711914
Optimization iteration: 29
----- Calculating Next Candidates Point From the Posterior PDF ... [COMPLETED]
	 * Accurac

	 * Accuracy of candidate # 1  is:  tensor(52.4071)
	 * Accuracy of candidate # 2  is:  tensor(54.2814)
	 * Accuracy of candidate # 3  is:  tensor(48.2898)
	 * Accuracy of candidate # 4  is:  tensor(50.9214)
	 * Accuracy of candidate # 5  is:  tensor(44.8452)
	 * Accuracy of candidate # 6  is:  